In [3]:
!pip install -q --upgrade google-generativeai langchain-google-genai

In [2]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [5]:
!pip install langchain_community

In [6]:
!pip install pypdf

In [7]:
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader('/content/ai.pdf')
loaded_pdf_doc = pdfloader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=['\n\n', '\n', ' '])

In [9]:
chunks = recursive_splitter.split_documents(loaded_pdf_doc)
print(len(chunks))

423


In [10]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.5 MB/s eta 0:

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001',google_api_key=API_KEY)

In [11]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [12]:
vector_index = vector_store.as_retriever(search_kwargs={'k':8})

In [13]:
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
    model,
    retriever = vector_index,
    return_source_documents=True
)

In [15]:
question = 'what is travel AI expert system according to the doc?'
response = rag({"query":question})
print(response)

{'query': 'what is travel AI expert system according to the doc?', 'result': 'According to the document, a travel AI expert system helps travelers plan and book trips. It interacts with users through a friendly interface (website, app, or chatbot) to gather travel dates, budget, and interests. The AI then uses this information to provide personalized recommendations and itineraries. It can also be integrated with booking platforms to enable travelers to make reservations directly through the system. The AI expert system draws from a wide knowledge base, potentially including the expertise of travel agents and other stakeholders.', 'source_documents': [Document(metadata={'trapped': '/False', 'source': '/content/ai.pdf', 'producer': 'Adobe PDF Library 17.0', 'creationdate': '2024-03-06T16:39:02+00:00', 'total_pages': 44, 'page': 21, 'page_label': '22', 'creator': 'Adobe InDesign 19.0 (Macintosh)', 'moddate': '2024-03-06T16:39:08+00:00'}, page_content='of rules and facts about a specific 